In [2]:
import tensorflow as tf
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, concatenate, LSTM, Lambda, TimeDistributed
from keras.models import Model
from keras.backend import expand_dims
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [23]:
#    -- Image PreProcessing Model --

# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
vid_input = Input(shape=(90, 150, 150, 3))

# First convolution extracts 4 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

model = TimeDistributed(Conv2D(4, (3,3), activation='relu'))(vid_input)
model = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(model)

# Second convolution extracts 5 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

model = TimeDistributed(Conv2D(5, (3,3), activation='relu'))(model)
model = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(model)

# Third convolution extracts 10 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

model = TimeDistributed(Conv2D(10, (3,3), activation='relu'))(model)
model = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(model)

model = TimeDistributed(Flatten())(model)

model = TimeDistributed(Dense(100, activation='relu'))(model)

model = LSTM(1)(model)
 
#    -- Audio PreProcessing Model --

aud_input = Input(shape=(1,))

# Put through a layer with linear activation
aud_model = Dense(1, activation='linear')(aud_input)

#     -- Concatenate Layers --
concatModel = concatenate([model, aud_model])
#concatModel = Lambda(lambda x: expand_dims(x, -1))(concatModel)

output = Dense(1, activation='softmax')(concatModel)

model = Model(inputs=[vid_input, aud_input], outputs=output)

model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['acc'])

print(model.summary())

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 90, 150, 150, 0                                            
__________________________________________________________________________________________________
time_distributed_65 (TimeDistri (None, 90, 148, 148, 112         input_17[0][0]                   
__________________________________________________________________________________________________
time_distributed_66 (TimeDistri (None, 90, 74, 74, 4 0           time_distributed_65[0][0]        
__________________________________________________________________________________________________
time_distributed_67 (TimeDistri (None, 90, 72, 72, 5 185         time_distributed_66[0][0]        
____________________________________________________________________________________________

In [24]:
# serialize model to JSON
model.save('model.h5')
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)